In [ ]:
import datetime
import requests
import json


response = requests.get('https://ofertahoraria.fi.uba.ar/api/last_update').json()

DATA = {
    "cuatrimestre": f"{response['currentQuarter']['year']}C{response['currentQuarter']['quarter']}",
    "timestamp": f"{response['lastUpdate']}",
}
DATA

In [ ]:
with open('src/data/horarios.js', 'r') as f:
    f.readline() # we remove the first `export const = ` line
    old_json = json.load(f)

old_data = {
            'cuatrimestre': old_json['cuatrimestre'],
            'timestamp': old_json['timestamp']
}

if (DATA is old_data): 
    # Nothing to update!
    exit()

In [ ]:
# Funcion para cuando ofertahoraria flashea y devuelve cualquier cosa
# r = requests.get(f'https://ofertahoraria.fi.uba.ar/api/...').json()
# response = decode_response(r)

def decode_response(dic):
    new_dic = {}
    for k,v in dic.items():        
        if isinstance(v, list):
            new_dic[k] = [decode_response(vv) for vv in v]
        elif isinstance(v, dict):
            new_dic[k] = decode_response(v)
            print(new_dic[k])
        else:
            new_dic[k] = str(v).encode('iso-8859-1', 'strict').decode("utf8", 'ignore')
    return new_dic

In [ ]:
response = requests.get('https://ofertahoraria.fi.uba.ar/api/departamentos').json()
departamentos = response.keys()
departamentos

In [ ]:
materias = []
for d in departamentos:
    response = requests.get(f'https://ofertahoraria.fi.uba.ar/api/departamento/{d}').json()
    if (response): 
        materias.extend(response.keys())
materias

In [ ]:
DATA["materias"] = []
DATA["cursos"] = []

DIAS = ["Domingo", "Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado"]

def format_clases(timetable):
    clases = []
    for clase in timetable:
        if clase["day"] == "Otro": continue
        clases.append({"dia": DIAS.index(clase["day"]),
                       "inicio": clase["from"],
                       "fin": clase["to"]})
    return clases

for m in materias:
    response = requests.get(f'https://ofertahoraria.fi.uba.ar/api/materia/{m}').json()
    if (response):
        comisiones = []
        for c in response['commisions']:
            clases = format_clases(c["timetable"])
            if not clases: continue
            comisiones.append({"codigo": f"{m}-{c['name']}", 
                               "materia": m,
                               "docentes": c["teachers"],
                               "clases": clases})
        
        if not comisiones: continue
        
        DATA["materias"].append({"codigo": m,
                                 "nombre": response["description"],
                                 "cursos": list(map(lambda x: x["codigo"], comisiones))})
        
        DATA["cursos"].extend(comisiones)
DATA

In [ ]:
dump = json.dumps(DATA, indent=2, ensure_ascii=False)        

with open('src/data/horarios.js', 'w') as fw:
    fw.write("export const data = ")
    fw.write("\n") # Do not remove me. Make me easy to parse.
    fw.write(dump)    

In [ ]:
### Para rehacer el archivo carreras.js (es un laburo muy chico como para automatizarlo)
### En un mundo ideal, no hay que hacerlo nunca, pero como hay cuatris donde se agregan materias, no esta mal chequearlo de vez en cuando

# Ir a 'horarios de cursada' del SIU, sin loggearse -> https://guaraniautogestion.fi.uba.ar/g3w/horarios_cursadas
# Poner los filtros de Responsable Academica (FIUBA), Propuesta (la ING a actualizar) y el Periodo Lectivo (el actual)

# Abrir una consola y buscar todos los codigos de las materias y copypastearlos en el archivo
# var docs = document.getElementsByClassName('accordion-toggle materia')
# var arr = [].slice.call(docs)
# arr.map(x=>x.innerHTML).map(x=>x.split('(')[1]).map(x=>x.split(')')[0]).map(x=>x.trim())

## Ojo: 
# - Agregar a ing de alimentos las materias de 'Año pulmon de ingenieria de alimentos'
# - Agregar a ing en agrimensura las materias de 'Agrimensura'
# - Agregar a ing naval y mecanica las materias de 'Ingenieria Naval'
# - El plan viejo de licenciatura se hizo medio a manopla entre lo que habia de cuatris pasados y el FIUBA Map
# Ojo con los duplicados!

### Para confirmar si alguna materia no esta contenida en ninguna carrera, clavar este codigo en algun lado de `useData.js`
## De todas formas, estas materias aparecen si no se selecciona ninguna carrera en el select del drawer
# React.useEffect(() => {
#     const allMateriasInCarreras = [];
#     jsonCarreras.forEach((c) => {
#       allMateriasInCarreras.push(...c.materias);
#     });
#     jsonData.materias.forEach((m) => {
#       if (!allMateriasInCarreras.includes(m.codigo)) {
#         alert(JSON.stringify(m));
#       }
#     });
#   }, []);

